# Урок 6. Кейс 2. Данные в страховом бизнесе. EDA.

** 1. Напишите текстом как минимум 6 функций распределения и соответствующие им функции правдоподобия.**

**Напишите, какие по вашему мнению целевые переменные из бизнеса (лучше - из страхового) соответствуют этим распределениям (по 2 примера на каждое).**

**Пример: биномиальное распределение для задачи моделирования пролонгации (продлил клиент полис на следующий период или нет).**

Равномерное - время наступление страхового случая.

$$
\begin{equation*}
f_X(x) = 
 \begin{cases}
   \frac{1}{b-a}, x\in{[a,b]}\\
   0, x\notin{[a,b]}
 \end{cases}
\end{equation*}
$$

Нормальное - время до первой страховой выплаты по договору.

$$
f(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

Биномиальное - приобритёт полис клиент в результате рекламной акции или нет.

$$
P(\{k\}) = C_n^kp^k(1-p)^{(n-k)}
$$

Геометрическое - количествово дней до поломки автомобиля.

$$
P(\{k\}) = (1-p)^{(k-1)}p
$$

**2. На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).**

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('datasets/freMPL-R.csv')

In [3]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

print('Dataset  Missing Variables')
for x in range(1,11):
    print(x,'\t',dct[x])

Dataset  Missing Variables
1 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
2 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
3 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
4 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
5 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
6 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
7 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
8 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass

In [4]:
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df59 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df59 = df59.drop(['Dataset'], axis=1)
df59 = df59.dropna(axis=1, how='all')
print('With duplicates\t\t', len(df59))
df59 = df59.drop_duplicates()
print('Without duplicates\t', len(df59))

With duplicates		 165200
Without duplicates	 115155


In [5]:
df = df59
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115155 entries, 145780 to 310979
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 1

In [6]:
# Вспомогательный столбец для суммирования числа полисов
df['PolicyCount'] = 1

# Вспомогательный столбец для суммирования числа полисов с убытками
df['ClaimCount'] = df['ClaimAmount'] > 0

# Вспомогательный столбец для суммирования числа полисов без убытков
df['NoClaimCount'] = df.PolicyCount - df.ClaimCount

df[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


Построим распределения частоты и среднего убытка от характера использования автомобиля.

In [7]:
df_group_veh_usage = df[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['VehUsage']).sum().reset_index()
df_group_veh_usage.head()

,VehUsage,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
0,Private,17385.293,38839,5.630104e+06,3066.0,35773
1,Private+trip to office,25945.001,59834,1.232871e+07,5812.0,54022
2,Professional,6574.757,14302,3.658391e+06,1718.0,12584
3,Professional run,977.100,2180,4.318442e+05,273.0,1907


In [8]:
fig = px.bar(df_group_veh_usage, x='VehUsage', y='PolicyCount', title='Распределение числа полисов по характеру использования автомобиля.', width=1000, height=400)
fig.show()

In [10]:
fig = go.Figure(data=[go.Bar(name='No Claims', x=df_group_veh_usage.VehUsage, y=df_group_veh_usage.NoClaimCount),
                      go.Bar(name='With Claims', x=df_group_veh_usage.VehUsage, y=df_group_veh_usage.ClaimCount)])
fig.update_layout(barmode='stack', xaxis_title='VehUsage', yaxis_title='PolicyCount',title='Распределение числа полисов по характеру использования автомобиля с убытками и без', width=1000, height=400)
fig.show()

Видим что наиболее часто клиенты используют авто для личных поездок и для поездок до работы. 

In [12]:
fig = go.Figure(data=go.Bar(name='No Claims', x=df_group_veh_usage.VehUsage, y=df_group_veh_usage.ClaimCount * 100 / df_group_veh_usage.NoClaimCount))
fig.update_layout(xaxis_title='VehUsage', yaxis_title='Percentage PolicyCount',title='Отношение числа полисов с убытками к полисам без убытков\nпо характеру использования автомобиля', width=1000, height=400)
fig.show()

Однако клиенты, который используют авто для профессиональной деятеольности чаще пользуются страховыми выплатами. Это может быть связано с отношением к автомобилю. Часто авто не принадлежит водителю, что сказывается на характере езды.